# CP 2024-25: Q2 Lecture 1 - Advanced programming

### General Guidelines

> ⚠️⚠️⚠️ READ CAREFULLY ⚠️⚠️⚠️

- Do not add, delete or create cells, write the answer only in the space marked with the three dots (`...`). Where function skeletons are provided, it is assumed that that function can be called again with different inputs somewhere else. So be careful to write code outside of functions.
  - Function should be ['pure'](https://en.wikipedia.org/wiki/Pure_function), thus no side effects, unless otherwise specified.
- Run the the first cell to import all libraries when opening the notebook before running your own code.
- Read carefully what is required to be printed/returned/plotted in the answer. Please do not output what is not asked for. 
  - If you used the print function for debugging, comment it out ( Ctlr + / ) before submitting
- All plots should have title, xlabel, ylabel, and legend (if there are more than one curve on the plot)
- Use the `help()` function, consult python documentation when using new functions, or do a web search and consult [stackoverflow](https://stackoverflow.com/questions/tagged/python)
- Please read the error messages if you get any, and try to understand what they mean. Debugging code is an essential skill to develop.
- You can use `%debug` to start an IPython console in a cell (or a scratchpad cell!) after an exception has occurred to try to debug.
- You can use `%pdb` to toggle the Python DeBugger (pdb) auto start after an unhandled exception.
- In the assignments you will find some tests put in place, to help you verify your solution. If these fail you are certain you did something wrong, thus look at the hints they provide. But passing these tests does __not__ mean your solution is actually correct.

Make sure you use `python3.12` and the package versions as stated in the provided `requirements.txt`. This file should also be on the course page.

In [1]:
# Importing relevant libraries in the assignment

# This will create static plots (no zooming etc.)
# otherwise try just plain `%matplotlib`, or install a backend such as ipympl or PyQt5 and
# do or `%matplotlib ipympl` `%matplotlib qt`
%matplotlib inline

REPEAT_IMPORTS = True

if REPEAT_IMPORTS or ("IMPORTED_ALL" not in globals()):  # To save you a bit of time

    def print_import_info(package):
        print(
            "Successfully imported %-15s \tVersion: %10s"
            % (package.__name__, package.__version__)
        )

    ### Standard library imports

    import sys

    print("Python version {}".format(sys.version))
    if sys.version_info < (3, 11):
        print(
            "\u001b[31m"  # red
            "\u001b[1m"  # bold
            "WARNING: Use Python 3.11 or newer to not encounter any errors or "
            "problems later on.\n"
            "\u001b[0m"  # reset
        )
    del sys  # Do not need it anymore

    import inspect
    import typing
    from typing import Callable, List, Tuple, Union

    ### Import third party libraries
    # Initialize self assessment helper
    import otter

    grader = otter.Notebook("Assignment_Q2_L1.ipynb")

    import numpy as np
    import numpy.typing as npt

    print_import_info(np)

    import scipy
    import scipy.integrate

    print_import_info(scipy)

    import matplotlib
    import matplotlib.pyplot as plt

    print_import_info(matplotlib)

    IMPORTED_ALL = True
    print("Finished importing packages")
else:
    print("Already imported all packages")

Python version 3.12.10 (v3.12.10:0cc81280367, Apr  8 2025, 08:46:59) [Clang 13.0.0 (clang-1300.0.29.30)]
Successfully imported numpy           	Version:      2.3.2
Successfully imported scipy           	Version:     1.16.1
Successfully imported matplotlib      	Version:     3.10.5
Finished importing packages


### Question 1: Defining classes
The Antoine equation is a mathematical expression used to describe the relationship between the vapor pressure of a pure substance and its temperature. It is commonly used in thermodynamics and chemical engineering to estimate the vapor pressure of liquids. Vapor pressures are crucial to calculate vapor-liquid equilibria, and therefore lie at the heart of many chemical engineering and process design problems.

The Antoine equation is given by:

$ \log_{10} P_{vap}(T) = A - \frac{B}{C + T} $

where:
- $ P_{vap} $ is the vapor pressure of the substance in mmHg.
- $ T $ is the temperature of the substance in °C.
- $ A $, $ B $, and $ C $ are substance-specific empirical constants.

In the first assignment of Q1, we used this equation to calculate several different vapor pressures for water. However, most chemical engineering design problems invlove multiple components with different thermodynamic properties. In our process of interest, we now also need to calculate the vapor pressures for methanol and ethanol.

If we want to keep track of all these Antoine-parameters in a simple script, our code is at high risk of becoming unreadable, unstructured, prone to errors, and messy. We want to manage this by writing a class called `AntoineComponent`, which encapsulates the Antoine parameters and the temperature validity domain for a chemical component. Implement this class below by following these steps:
- Declare the class `AntoineComponent`.
- Implement the constructor, that takes the following arguments: `(A: float, B: float, C: float, T_min: float, T_max: float)`. All arguments of the constructor are to be stored as attributes of the class.
- Implement a method `calculate_vapor_pressure`, which takes only (!) the temperature as an input. Use the antoine parameters that are stored in the attributes for calculation.
- Implement a mechanism in `calculate_vapor_pressure` that returns `-1` when the temperature argument is outside the component's validity domain.

In [6]:
class AntoineComponent:
    """
    A class to represent the various components which are required to compute the Antoine equation.
    
    Attributes:
        A       (float): parameter A
        B       (flaot): Parameter B
        C       (flaot): Parameter C
        T_min   (flaot): Minimum temperature  
        T_max   (flaot): Maximum temperature
    
    Methods:
        Computes the vapore pressure of the substance
    """

    def __init__(self,A:float,B:float,C:float,T_min:float, T_max:float):
        """
        Initializes a new AntoineComponent instance with its characteristic constants.

        Args:
            A (float): Antoine constant A.
            B (float): Antoine constant B.
            C (float): Antoine constant C.
            T_min (float): Minimum valid temperature for the equation (°C).
            T_max (float): Maximum valid temperature for the equation (°C).


        """
        self.A = A
        self.B = B
        self.C = C
        self.T_min = T_min
        self.T_max = T_max
    
    def calculate_vapor_pressure(self,T:float)->float:
        """The mechanism which calculates the vapor pressure of of the substance.
        Returns a -1 if the temperature is outside the temperature range. """

        if T<self.T_min or T>self.T_max:
            return -1

        vap_pressure = 10**(self.A-self.B/(self.C+T))
        return vap_pressure


In [7]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2: Create objects by instantiating classes
Now, it's time to fill our new class with some life! Create instances of the the AntoineComponent class for `water`, `ethanol`, and `methanol`, and calculate yourself some vapor pressures!

- For water, in the temperature range 1°C to 100°C, the Antoine parameters are:
  - $ A = 8.07131 $
  - $ B = 1730.63 $
  - $ C = 233.426 $
- For ethanol, in the temperature range of -57°C and 80°C the Antoine parameters are:
  - $ A = 8.20417 $
  - $ B = 1642.89 $
  - $ C = 230.300 $
- For methanol, in the temperature range of 15°C and 100°C the Antoine parameters are:
  - $ A = 8.08097 $
  - $ B = 1582.27 $
  - $ C = 239.7 $

In [8]:
water = AntoineComponent(8.07131,1730.63,233.426,1.0,100.0)
ethanol = AntoineComponent(8.20417,1642.89,230.300,-57.0,80.0)
methanol = AntoineComponent(8.08097,1582.27,239.7,15.0,100.0)

print(water.calculate_vapor_pressure(12))
print(water.calculate_vapor_pressure(37))
print(ethanol.calculate_vapor_pressure(12))
print(methanol.calculate_vapor_pressure(65))

10.465864743433116
46.95333822686567
26.53223265959557
772.8438567333864


In [9]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3: Unit testing
Antoine's equation can be used with Raoult's law to determine coexisting vapor and liquid compositions, assuming an ideal mixture. Raoult's law is given in the equation below for reference.

$P_i = y_i \cdot P_{\text{total}} = x_i \cdot P_i^*$

where:

- $ P_i $ is the partial pressure of component $ i $ in the vapor phase.
- $ y_i $ is the mole fraction of component $ i $ in the vapor phase.
- $ P_{\text{total}} $ is the total pressure of the system.
- $ x_i $ is the mole fraction of component $ i $ in the liquid phase.
- $ P_i^* $ is the vapor pressure of the pure component $ i $ at the given temperature, which can be calculated with Antoine's equation.

The function `vapor_fraction` below uses Antoine's equation and Raoult's law to calculate the coexisting vapor composition of a species in a liquid at given conditions. For this, it takes the following inputs: 
- The component's Antoine-paramters (`A`, `B`, and `C`) 
- The temperature validity range (`T_min` and `T_max`) of the Antoine parameters
- The system's temperature (in C) and total pressure (in Pa) (`T` and `P_total`)
- The component's liquid-phase mole fraction (`x_i`). 
The output is the coexisting vapor-phase mole fraction `y_i` of the component.

Write some tests to ensure the correct functionality of the function. Then, run your tests by either calling them manually, or by installing and using the ipytest package. ([ipytest Documentation](https://pypi.org/project/ipytest/))

In [10]:
def vapor_fraction(
    A: float,
    B: float,
    C: float,
    T_min: float,
    T_max: float,
    T: float,
    P_total: float,
    x_i: float,
) -> float:
    """
    Calculates the coexisting vapor-phase mole fraction of a species in a liquid at given conditions
    using Antoine's equation and Raoult's law.

    Args:
        A (float): Antoine parameter A for the species.
        B (float): Antoine parameter B for the species.
        C (float): Antoine parameter C for the species.
        T_min (float): Minimum temperature for the validity of Antoine's equation (in Celsius).
        T_max (float): Maximum temperature for the validity of Antoine's equation (in Celsius).
        T (float): System temperature in Celsius.
        P_total (float): System pressure in Pa.
        x_i (float): Liquid-phase mole fraction of the species.

    Returns:
        float: Vapor-phase mole fraction y_i of the species. If the temperature
        is outside the validity domain, the return value is -1.

    """
    # Check if temperature is within the validity range
    if not (T_min <= T <= T_max):
        return -1

    # Calculate the vapor pressure using Antoine's equation
    # Convert from mmHg to Pa by multiplying by 133.322
    P_i_star = 10 ** (A - B / (C + T)) * 133.322

    # Apply Raoult's law to find partial pressure
    P_i = x_i * P_i_star

    # Calculate vapor-phase mole fraction
    y_i = P_i / P_total

    return y_i

### Question 3.1
Write a test for water called `test_vapor_fraction_water` using the Antoine parameters of the previous question. First, make sure that the function correctly returns `-1` if the temperature is out of bounds. Write an assert statement for a temperture below the permitted temperature range, and one assert statement for a temperature above the permitted temperature range. Then, test the function for water at a system pressure of 150 000 Pa, a temperature of 80°C and a liquid water fraction of 0.6. The vapor fraction should calculate to ~0.189 (tolerance 0.001). (Your test will have three `assert` statements)


In [ ]:
# open terminal and run pip install ipytest
# this will install the test running package, you can confirm the installation of this package by running either of the following
# pip list 
import ipytest

def test():
    # obtain value which is to be assessed
    value = vapor_fraction()
    assert 


In [ ]:
grader.check("q3.1")

### Question 3.2
Write a test for water called `test_vapor_fraction_ethanol` using the Antoine parameters of the previous question. First, make sure that the function correctly returns `-1` if the temperature is out of bounds. Write an assert statement for a temperture below the permitted temperature range, and one assert statement for a temperature above the permitted temperature range. Then, test the function for ethanol at a system pressure of 120 000 Pa, a temperature of 60°C and a liquid ethanol fraction of 0.3. The vapor fraction should calculate to ~0.117 (tolerance 0.001). (Your test will have three `assert` statements)

In [ ]:
...

In [ ]:
grader.check("q3.2")

### Question 4: Single-responsibility principle
The function `vapor_fraction` is in clear violation of the single-responsibility principle. It calculates both the vapor pressure with Antoine's equation and the vapor fraction with Raoult's law. Now, it is not possible to use the functionality apart from each other, and it is harder to identify which part of the code does what.

Rewrite the function `vapor_fraction` and split up the functionality into smaller code elements.  Feel free to make use of your class `AntoineComponent` from question 1, or write new functions or classes. Because the function may already be used by other parts of code, make sure that the function header remains the same.

This question has no autograding tests. Use your own tests from Question 3 to see if the function still works as before!


In [ ]:
...

### Question 5: Object-oriented programming in practice
In practice, it takes some experience to know what data entities to model as classes or objects in object-oriented programming. Often, it depends on requirements and what you want to do. Therefore, it helps to think about this in the context of chemical engineering.

Below is a list of concepts from chemical engineering. 
- A heat exchanger
- A temperature measurement (So, a single measurement of temperature at a given location at a specified time)
- A chemical reaction
- A flowrate in kmol/hr in a pipe
- A permeability coefficient in a membrane unit
- The anode material in an electrolyzer
- A process flow diagram

For each of the concepts, discuss the questions below. Feel free to exchange your ideas with a colleague:
- Would you model this as a class and with objects? Why (not)?
- What would this class need to store as attributes? Would these be basic data types like floats and strings? Would they be numpy arrays? Or maybe even objects of another class? Which of the attributes are only used internally by methods in the class (private attributes), and which should be accessible from outside the class (public attributes/properties)?
- What would you want your class to be able to do? What should it calculate? Which of these calculations are only used internally by other methods in the class (private methods), and which should be called from outside the class (public methods)?
- Can you think of how the answer to the above questions might change with the context? E.g. if you are tasked to write a simple process simulation program vs. an automatic cost estimation program?

### Bonus Question: Sick of jupyter notebook? Making a small graphical interface with Streamlit.
Python offers several packages for building graphical user interfaces (GUIs). These GUIs make applications more intuitive and user-friendly by allowing users to interact with them visually rather than through code. A popular library for this is Streamlit, which excels at easily and quickly making interactive web apps for data science and machine learning projects. It transforms Python scripts into shareable web applications with a simple, intuitive API for displaying data, visualizations, and user inputs.

Make a small application for Antoine data in Streamlit. You could include a feature that calculates vapor pressure at the click of a button, or directly plots the vapor pressure over a temperature range. Here is some helpful material to get you started:
- [An excellent youtube tutorial with the basics of streamlit and how to build your first application](https://www.youtube.com/watch?v=D0D4Pa22iG0&t=41s)
- [The streamlit documentation page with more detailed setup instructions and description of the streamlit functionality](https://docs.streamlit.io/)

Note: Streamlit does not work well from a jupyter notebook. We recommend to set this up in a separate Python project, copy over the code from this assignment into Python modules, and build your streamlit application there!